In [36]:
import os
import tweepy as tw
import pandas as pd
import geocoder 
import numpy as np 
import json
from urllib.request import urlopen
import reverse_geocoder as rg
import math 
from scipy import * 
import geopip 
import datetime 

In [2]:

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [3]:
new_search = "ban+islam" 

tweets = tw.Cursor(api.search,
                   q=new_search,
                   since='2020-01-01').items(1000)



In [4]:
users_locs = [[tweet.user.screen_name, tweet.user.location, tweet.coordinates, tweet.user.lang, tweet.text] for tweet in tweets]
users_locs

[['Yazowyy',
  '',
  None,
  None,
  "RT @a_dawg_thoughts: it's always screw France, but today especially.. y'all really just hate Islam and Muslims don't you? Was the uni hijab…"],
 ['Am10Mary',
  'Kuwait',
  None,
  None,
  '@faatemsh They give me Middle Ages vibes France statements (towards Islam) is a Joke, \nthey be like u have the righ… https://t.co/otNGyOpaAX'],
 ['coolsandira',
  'India',
  None,
  None,
  'Requesting Modi Government To Ban Shariat Law Of Islam - Save Innocent Muslims From Terrorism\n\n#banshariat… https://t.co/whaHyw5FNS'],
 ['Swastikagrawa14',
  '',
  None,
  None,
  '@being_caravan Bas abhi Quran padhnea chod dea islam chod dea human ban jayegea'],
 ['C71778967',
  '',
  None,
  None,
  '@nYoperaguy @WakaFlocka AC: "is there a war between the west &amp; radical islam, or.." \nDT: "it [islam] is very radica… https://t.co/sxLyskfA73'],
 ['manibunny1',
  'Hellas',
  None,
  None,
  'RT @agelaw_7: TIME TO TAKE OUR COUNTRIES BACK!! \n#BAN_ISLAM'],
 ['Falcon_AlHin

In [5]:
tweet_text = pd.DataFrame(data=users_locs, 
                    columns=['user', "location", "coordinates", "language", "text"])
tweet_text 

,user,location,coordinates,language,text
0,Yazowyy,,None,None,"RT @a_dawg_thoughts: it's always screw France,..."
1,Am10Mary,Kuwait,None,None,@faatemsh They give me Middle Ages vibes Franc...
2,coolsandira,India,None,None,Requesting Modi Government To Ban Shariat Law ...
3,Swastikagrawa14,,None,None,@being_caravan Bas abhi Quran padhnea chod dea...
4,C71778967,,None,None,"@nYoperaguy @WakaFlocka AC: ""is there a war be..."
...,...,...,...,...,...
995,IniPastiDotCom,"Makassar, Indonesia",None,None,Prodi Administrasi Bisnis Universitas Islam Ma...
996,HerminePatriote,,None,None,Puisqu’il est vraisemblablement plus malvenu d...
997,manojshah65,,None,None,RT @pulindsamant: Promise by @JoeBiden to Amer...
998,devnamdev99,"Pindwara, India",None,None,@MdSamiHaider @beatty_coretta RSS will never b...


In [14]:
tweet_text['latitude'] = 0
tweet_text['longitude'] = 0


def get_coordinates(row): 
    result = geocoder.arcgis(row.location)
    row.latitude = result.x
    row.longitude = result.y
    return row 

df = tweet_text.apply(get_coordinates, axis=1)

    

In [15]:
df

,user,location,coordinates,language,text,latitude,longitude
0,Yazowyy,,None,None,"RT @a_dawg_thoughts: it's always screw France,...",NaN,NaN
1,Am10Mary,Kuwait,None,None,@faatemsh They give me Middle Ages vibes Franc...,47.612166,29.355337
2,coolsandira,India,None,None,Requesting Modi Government To Ban Shariat Law ...,79.443327,23.379380
3,Swastikagrawa14,,None,None,@being_caravan Bas abhi Quran padhnea chod dea...,NaN,NaN
4,C71778967,,None,None,"@nYoperaguy @WakaFlocka AC: ""is there a war be...",NaN,NaN
...,...,...,...,...,...,...,...
995,IniPastiDotCom,"Makassar, Indonesia",None,None,Prodi Administrasi Bisnis Universitas Islam Ma...,119.422930,-5.146500
996,HerminePatriote,,None,None,Puisqu’il est vraisemblablement plus malvenu d...,NaN,NaN
997,manojshah65,,None,None,RT @pulindsamant: Promise by @JoeBiden to Amer...,NaN,NaN
998,devnamdev99,"Pindwara, India",None,None,@MdSamiHaider @beatty_coretta RSS will never b...,73.045640,24.792520


In [18]:
df["country"] = np.nan


def get_iso(row): 
    if(not math.isnan(row.latitude) and not math.isnan(row.longitude)): 
        c = [(row.longitude,row.latitude)] 
        country = rg.search(c)[0].get('cc')
        row.country = country
    return row 

#c = (51.5214588,-0.1729636)
#code = rg.search(c)[0].get('cc')
#print(code[0])
df2 = df.apply(get_iso, axis=1)

df2

,user,location,coordinates,language,text,latitude,longitude,country
0,Yazowyy,,None,None,"RT @a_dawg_thoughts: it's always screw France,...",NaN,NaN,NaN
1,Am10Mary,Kuwait,None,None,@faatemsh They give me Middle Ages vibes Franc...,47.612166,29.355337,KW
2,coolsandira,India,None,None,Requesting Modi Government To Ban Shariat Law ...,79.443327,23.379380,IN
3,Swastikagrawa14,,None,None,@being_caravan Bas abhi Quran padhnea chod dea...,NaN,NaN,NaN
4,C71778967,,None,None,"@nYoperaguy @WakaFlocka AC: ""is there a war be...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
995,IniPastiDotCom,"Makassar, Indonesia",None,None,Prodi Administrasi Bisnis Universitas Islam Ma...,119.422930,-5.146500,ID
996,HerminePatriote,,None,None,Puisqu’il est vraisemblablement plus malvenu d...,NaN,NaN,NaN
997,manojshah65,,None,None,RT @pulindsamant: Promise by @JoeBiden to Amer...,NaN,NaN,NaN
998,devnamdev99,"Pindwara, India",None,None,@MdSamiHaider @beatty_coretta RSS will never b...,73.045640,24.792520,IN


In [20]:
df2.to_csv('hate_speech.csv', index = False)

In [113]:

tweets2 = tw.Cursor(api.search,
                   q=new_search,
                   geocode = "39.8283,-98.5795,1400mi").items(1000)



In [114]:
users_locs2 = [[tweet.user.id,  
                tweet.user.screen_name,
                tweet.user.name,
                tweet.id, 
                tweet.created_at, 
                tweet.user.location, 
                tweet.coordinates, 
                tweet.user.lang, 
                tweet.text] for tweet in tweets2]
users_locs2

[[4801809980,
  'gilchriste76',
  'Gilchrist 💧🔥🏊\u200d♂️🤑🌾🐨🛂 Angus = 🧛\u200d♂️',
  1320158423963496448,
  datetime.datetime(2020, 10, 25, 0, 21, 10),
  'Global citizen',
  None,
  None,
  "RT @SBSNews: .@TheFeedSBS has seen several Facebook posts and emails shared by Andrew McNabb's accounts that call to ban Muslim immigration…"],
 [920008209338978305,
  'No1Captain',
  'Peter C. Pro Tory but how long? 🇬🇧🇭🇺🇺🇲',
  1320145426746646529,
  datetime.datetime(2020, 10, 24, 23, 29, 31),
  'Earth',
  None,
  None,
  '@debatenothate2 at the risk if a Twitter ban I say add Islam.'],
 [778705812172574720,
  'Peter08374648',
  'MPeterN',
  1320121442034454528,
  datetime.datetime(2020, 10, 24, 21, 54, 13),
  'Washington, DC',
  None,
  None,
  'RT @WithinSyriaBlog: Today HTS media is attacking Paris and accusing its of waging a war on Islam and Prophet Mohamad AS, the group even is…'],
 [1883754805,
  'Yazowyy',
  'Ü',
  1320104872059019265,
  datetime.datetime(2020, 10, 24, 20, 48, 22),
  '',
  N

In [115]:
tweet_text2 = pd.DataFrame(data=users_locs2, 
                    columns=['userID', "screen_name", "name", "tweetID", "create_dt", "location", "coordinates", "language", "text"])
tweet_text2

,userID,screen_name,name,tweetID,create_dt,location,coordinates,language,text
0,4801809980,gilchriste76,Gilchrist 💧🔥🏊‍♂️🤑🌾🐨🛂 Angus = 🧛‍♂️,1320158423963496448,2020-10-25 00:21:10,Global citizen,None,None,RT @SBSNews: .@TheFeedSBS has seen several Fac...
1,920008209338978305,No1Captain,Peter C. Pro Tory but how long? 🇬🇧🇭🇺🇺🇲,1320145426746646529,2020-10-24 23:29:31,Earth,None,None,@debatenothate2 at the risk if a Twitter ban I...
2,778705812172574720,Peter08374648,MPeterN,1320121442034454528,2020-10-24 21:54:13,"Washington, DC",None,None,RT @WithinSyriaBlog: Today HTS media is attack...
3,1883754805,Yazowyy,Ü,1320104872059019265,2020-10-24 20:48:22,,None,None,"RT @a_dawg_thoughts: it's always screw France,..."
4,1003274852294279168,Falcon_AlHind,Shaheen,1320085835182530561,2020-10-24 19:32:43,🍌Republic,None,None,"-Macron with his ""Islam is in Criss"" &amp; ong..."
...,...,...,...,...,...,...,...,...,...
136,152230763,WhyMehrab,Mehrab Chandio,1317187044683251712,2020-10-16 19:33:58,United States,None,None,he wanted to Ban #Islam: Look what happened: ...
137,753694909,Peppezappulla,giuseppe zappulla,1317186146170961920,2020-10-16 19:30:24,"Siracusa, Sicilia",None,None,RT @Comemigirano: @Carlo_bis @GAcquistapace @P...
138,362199127,keithwmca,Keithwmca,1317156237419663363,2020-10-16 17:31:33,"Dallas, Texas",None,None,@EricTrump @realDonaldTrump Just #Christians t...
139,122733775,Comemigirano,"#Maverick, #4NonniItaliani, gli altri... vedremo",1317092358345863168,2020-10-16 13:17:43,"Winter Park, FL USA",None,None,@Carlo_bis @GAcquistapace @Peppezappulla @leop...


In [100]:
tweet_text2.create_dt.sort_values()

139   2020-10-16 13:14:22
138   2020-10-16 13:17:43
137   2020-10-16 17:31:33
136   2020-10-16 19:30:24
135   2020-10-16 19:33:58
              ...        
4     2020-10-24 16:51:10
3     2020-10-24 19:32:43
2     2020-10-24 20:48:22
1     2020-10-24 21:54:13
0     2020-10-24 23:29:31
Name: create_dt, Length: 140, dtype: datetime64[ns]

In [101]:
tweet_text2['latitude'] = 0
tweet_text2['longitude'] = 0


def get_coordinates(row): 
    result = geocoder.arcgis(row.location)
    row.latitude = result.x
    row.longitude = result.y
    return row 

df4 = tweet_text2.apply(get_coordinates, axis=1)

In [102]:
df4["country"] = np.nan
df4["admin1"] = np.nan
df4["admin2"] = np.nan


def get_iso(row): 
    if(not math.isnan(row.latitude) and not math.isnan(row.longitude)): 
        c = [(row.longitude,row.latitude)] 
        data = rg.search(c)[0]
        country = data.get('cc')
        row.country = country
        row.admin1 = data.get('admin1')
        row.admin2 = data.get('admin2')
    return row 


df5 = df4.apply(get_iso, axis=1)

df5

,userID,screen_name,name,tweetID,create_dt,location,coordinates,language,text,latitude,longitude,country,admin1,admin2
0,920008209338978305,No1Captain,Peter C. Pro Tory but how long? 🇬🇧🇭🇺🇺🇲,1320145426746646529,2020-10-24 23:29:31,Earth,None,None,@debatenothate2 at the risk if a Twitter ban I...,-102.410200,34.232940,US,Texas,Lamb County
1,778705812172574720,Peter08374648,MPeterN,1320121442034454528,2020-10-24 21:54:13,"Washington, DC",None,None,RT @WithinSyriaBlog: Today HTS media is attack...,-77.031960,38.890370,US,"Washington, D.C.",
2,1883754805,Yazowyy,Ü,1320104872059019265,2020-10-24 20:48:22,,None,None,"RT @a_dawg_thoughts: it's always screw France,...",NaN,NaN,NaN,NaN,NaN
3,1003274852294279168,Falcon_AlHind,Shaheen,1320085835182530561,2020-10-24 19:32:43,🍌Republic,None,None,"-Macron with his ""Islam is in Criss"" &amp; ong...",-93.480810,37.126260,US,Missouri,Greene County
4,1280235828833288195,canthama1,Canthama1,1320045177784893444,2020-10-24 16:51:10,,None,None,RT @WithinSyriaBlog: Today HTS media is attack...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,152230763,WhyMehrab,Mehrab Chandio,1317187044683251712,2020-10-16 19:33:58,United States,None,None,he wanted to Ban #Islam: Look what happened: ...,-99.414619,39.398703,US,Kansas,Rooks County
136,753694909,Peppezappulla,giuseppe zappulla,1317186146170961920,2020-10-16 19:30:24,"Siracusa, Sicilia",None,None,RT @Comemigirano: @Carlo_bis @GAcquistapace @P...,15.281330,37.078010,IT,Sicily,Provincia di Siracusa
137,362199127,keithwmca,Keithwmca,1317156237419663363,2020-10-16 17:31:33,"Dallas, Texas",None,None,@EricTrump @realDonaldTrump Just #Christians t...,-96.795120,32.778220,US,Texas,Dallas County
138,122733775,Comemigirano,"#Maverick, #4NonniItaliani, gli altri... vedremo",1317092358345863168,2020-10-16 13:17:43,"Winter Park, FL USA",None,None,@Carlo_bis @GAcquistapace @Peppezappulla @leop...,-81.350710,28.594570,US,Florida,Orange County


In [103]:
df5.rename(columns={"latitude": "longitude", "longitude": "latitude"})

,userID,screen_name,name,tweetID,create_dt,location,coordinates,language,text,longitude,latitude,country,admin1,admin2
0,920008209338978305,No1Captain,Peter C. Pro Tory but how long? 🇬🇧🇭🇺🇺🇲,1320145426746646529,2020-10-24 23:29:31,Earth,None,None,@debatenothate2 at the risk if a Twitter ban I...,-102.410200,34.232940,US,Texas,Lamb County
1,778705812172574720,Peter08374648,MPeterN,1320121442034454528,2020-10-24 21:54:13,"Washington, DC",None,None,RT @WithinSyriaBlog: Today HTS media is attack...,-77.031960,38.890370,US,"Washington, D.C.",
2,1883754805,Yazowyy,Ü,1320104872059019265,2020-10-24 20:48:22,,None,None,"RT @a_dawg_thoughts: it's always screw France,...",NaN,NaN,NaN,NaN,NaN
3,1003274852294279168,Falcon_AlHind,Shaheen,1320085835182530561,2020-10-24 19:32:43,🍌Republic,None,None,"-Macron with his ""Islam is in Criss"" &amp; ong...",-93.480810,37.126260,US,Missouri,Greene County
4,1280235828833288195,canthama1,Canthama1,1320045177784893444,2020-10-24 16:51:10,,None,None,RT @WithinSyriaBlog: Today HTS media is attack...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,152230763,WhyMehrab,Mehrab Chandio,1317187044683251712,2020-10-16 19:33:58,United States,None,None,he wanted to Ban #Islam: Look what happened: ...,-99.414619,39.398703,US,Kansas,Rooks County
136,753694909,Peppezappulla,giuseppe zappulla,1317186146170961920,2020-10-16 19:30:24,"Siracusa, Sicilia",None,None,RT @Comemigirano: @Carlo_bis @GAcquistapace @P...,15.281330,37.078010,IT,Sicily,Provincia di Siracusa
137,362199127,keithwmca,Keithwmca,1317156237419663363,2020-10-16 17:31:33,"Dallas, Texas",None,None,@EricTrump @realDonaldTrump Just #Christians t...,-96.795120,32.778220,US,Texas,Dallas County
138,122733775,Comemigirano,"#Maverick, #4NonniItaliani, gli altri... vedremo",1317092358345863168,2020-10-16 13:17:43,"Winter Park, FL USA",None,None,@Carlo_bis @GAcquistapace @Peppezappulla @leop...,-81.350710,28.594570,US,Florida,Orange County


In [104]:
df5.to_csv('hate_speech_oct_16_24_US.csv', index = False)
